In [2]:
import pandas as pd 
import os

In [144]:
# global variables shit
min_n, max_n = 5, 8

In [177]:
gen = os.walk('SymbTr/txt')

paths = [(a, b, c) for a, b, c in gen]
paths = [ (a,c) for a,b,c in paths][0]
paths = [path for path in paths[1]]

makam_dict = {}

for path in paths[1]:
    makam = path.split('--')[0]
    if makam in makam_dict:
        makam_dict[makam] += 1
    else:
        makam_dict[makam] = 1

for m in makam_dict:
    if makam_dict[m] > 19:
        print(m, makam_dict[m])

In [210]:
makam_dict = {'rast', 'ussak'}
paths_new = {}

for makam in makam_dict:
    counter = 0
    paths_list = []
    for path in paths:
        if path.split('--')[0] == makam:
            paths_list.append(path)
            counter += 1
        if counter == 3:
            paths_new[makam] = paths_list
            break
paths_new





{'ussak': ['ussak--yuruksemai--yuruksemai--dila_cunem--mahmut_celebi.txt',
  'ussak--turku--sofyan--telgirafin_tellerine-2.txt',
  'ussak--turku--nimsofyan--ayva_cicek--.txt'],
 'rast': ['rast--turku--nimsofyan--sira_sira--.txt',
  'rast--sarki--musemmen--vuslatindan_gayri--haci_arif_bey.txt',
  'rast--sarki--senginsemai--sevda_ile_dillendi--munir_nurettin_selcuk.txt']}

In [120]:
def get_relative_intervals(list_of_tuples):
    lt = [(0, list_of_tuples[0][1])]
    for i in range(1, len(list_of_tuples)):
        lt.append((list_of_tuples[i][0] - list_of_tuples[i - 1][0], list_of_tuples[i][1]))
    return lt

In [126]:
def get_ngrams_from_list_of_tuples(list_of_notes, min_n, max_n):
    len_list = len(list_of_notes)
    if len_list < min_n:
        return {}
    max_n = min(max_n, len_list)
    d = {}
    for n in range(min_n, max_n + 1):
        d_n = {}
        for m in range(0, len_list - n + 1):
            relative_intervals = str(get_relative_intervals(list_of_notes[m : m + n]))
            if relative_intervals in d_n:
                d_n[relative_intervals] += 1
            else:
                d_n[relative_intervals] = 1
        d[n] = d_n
    return d

testing = [(358, (1, 32)), (349, (1, 32)), (358, (1, 32)), (367, (1, 32)), (358, (1, 16)), (349, (1, 16)), (344, (1, 4)), (349, (1, 32)), (344, (1, 32)), (349, (1, 32)), (358, (1, 32)), (349, (1, 16)), (344, (1, 16)), (336, (3, 8)), (343, (1, 32)), (336, (1, 32)), (343, (1, 32)), (349, (1, 32)), (343, (1, 16)), (336, (1, 16)), (327, (1, 8))]

get_ngrams_from_list_of_tuples(testing, min_n, max_n)

{3: {'[(0, (1, 32)), (-9, (1, 32)), (9, (1, 32))]': 1,
  '[(0, (1, 32)), (9, (1, 32)), (9, (1, 32))]': 1,
  '[(0, (1, 32)), (9, (1, 32)), (-9, (1, 16))]': 2,
  '[(0, (1, 32)), (-9, (1, 16)), (-9, (1, 16))]': 1,
  '[(0, (1, 16)), (-9, (1, 16)), (-5, (1, 4))]': 1,
  '[(0, (1, 16)), (-5, (1, 4)), (5, (1, 32))]': 1,
  '[(0, (1, 4)), (5, (1, 32)), (-5, (1, 32))]': 1,
  '[(0, (1, 32)), (-5, (1, 32)), (5, (1, 32))]': 1,
  '[(0, (1, 32)), (5, (1, 32)), (9, (1, 32))]': 1,
  '[(0, (1, 32)), (-9, (1, 16)), (-5, (1, 16))]': 1,
  '[(0, (1, 16)), (-5, (1, 16)), (-8, (3, 8))]': 1,
  '[(0, (1, 16)), (-8, (3, 8)), (7, (1, 32))]': 1,
  '[(0, (3, 8)), (7, (1, 32)), (-7, (1, 32))]': 1,
  '[(0, (1, 32)), (-7, (1, 32)), (7, (1, 32))]': 1,
  '[(0, (1, 32)), (7, (1, 32)), (6, (1, 32))]': 1,
  '[(0, (1, 32)), (6, (1, 32)), (-6, (1, 16))]': 1,
  '[(0, (1, 32)), (-6, (1, 16)), (-7, (1, 16))]': 1,
  '[(0, (1, 16)), (-7, (1, 16)), (-9, (1, 8))]': 1},
 4: {'[(0, (1, 32)), (-9, (1, 32)), (9, (1, 32)), (9, (1, 32))]'

In [15]:
path_to_txt = 'SymbTr/txt/acemasiran--aranagme--agiraksak----001.txt'
path_to_txt = 'SymbTr/txt/ferahnak--sarki--aksak--bir_nigahin--tanburi_cemil_bey.txt'

0      358
1      349
2      358
3      367
4      358
5      349
6      344
7      349
8      344
9      349
10     358
11     349
12     344
13     336
14     343
15     336
16     343
17     349
18     343
19     336
20     327
21      -1
22     336
23     327
24     336
25     340
26     336
27     327
28     327
29     318
      ... 
487    314
488    322
489    314
490    305
491    305
492    305
493    314
494    322
495    327
496    322
497    327
498    322
499    314
500     -1
501    305
502    296
503    291
504    336
505    327
506    327
507    305
508    296
509    291
510    286
511    291
512    296
513    305
514    296
515    291
516    291
Name: Koma53, Length: 517, dtype: int64


[(0, (1, 1)), (10, (4, 4)), (-10, (1, 2))]

In [252]:
# this part takes in a sing song, breaks it up into the sections between rests




def get_ngram_dict_from_score(path, min_n, max_n):
    score = pd.read_csv(path, delimiter = '\t')
    data = score[['Payda', 'Pay', 'Koma53']]
    list_of_shit = []
    current_list = []

    for row in data.iterrows():
        k = row[1]['Koma53']
        p = row[1]['Pay']
        d = row[1]['Payda']

        if k == -1 and current_list:
            list_of_shit.append(current_list)
            current_list = []
        else:
            current_list.append((k, (p, d)))

    if current_list:
        list_of_shit.append(current_list)
    
    big_dic = {}
    for i in range(min_n, max_n + 1):
        big_dic[i] = {}

    for shit in list_of_shit:
        mini_dic = get_ngrams_from_list_of_tuples(shit, min_n, max_n)
        for n in range(min_n, max_n + 1):
            if n not in mini_dic:
                mini_dic[n] = {}
            big_dic[n] = dict((i, big_dic[n].get(i, 0) + mini_dic[n].get(i, 0)) for i in set(big_dic[n])|set(mini_dic[n]))
    
    return big_dic

path = 'SymbTr/txt/ferahnak--sarki--aksak--bir_nigahin--tanburi_cemil_bey.txt'


#ngram_dict = get_ngram_dict_from_score(path, min_n, max_n)
    

In [242]:
get_ngrams_from_list_of_tuples([(296, (1, 8)), (291, (1, 8)), (283, (1, 4)), (291, (1, 4)), (296, (1, 2))], 5, 8

{5: {'[(0, (1, 8)), (-5, (1, 8)), (-8, (1, 4)), (8, (1, 4)), (5, (1, 2))]': 1}}

In [290]:
dict_of_makam_dicts = {}

for makam_name in paths_new:
    d = {}
    for path in paths_new[makam_name]:
        path = 'SymbTr/txt/' + path 
        d[path] = get_ngram_dict_from_score(path, min_n, max_n)
    dict_of_makam_dicts[makam_name] = d

dict_of_makam_ngrams = {}

for makam_name in dict_of_makam_dicts:
    d = {}
    for i in range(min_n, max_n + 1):
        d[i] = {}
    for score in dict_of_makam_dicts[makam_name]:
        s = dict_of_makam_dicts[makam_name][score]
        for i in range(min_n, max_n + 1):
            d[i] = dict((j, d[i].get(j, 0) + s[i].get(j, 0)) for j in set(s[i]) | set(d[i]))
    dict_of_makam_ngrams[makam_name] = d




2
